In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from sklearn.metrics import f1_score
from tensorflow.keras.applications.vgg16 import preprocess_input

from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from os import *
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap=plt.cm.Blues) #'viridis'
    plt.savefig('/home/jovyan/img1.png')
    plt.show()
    
    return conf_matx

In [4]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy; test='+ str(np.round(model_history.history['val_accuracy'][-1], 3)),
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    plt.savefig('/home/jovyan/curve.png')
    plt.show()
    
    return fig

In [5]:
def resize(x):
    rescaled = []

    for i in x:

        scale_percent = 140 # percent of original size
        width = int(i.shape[1] / (scale_percent / 100))
        height = int(i.shape[0] / (scale_percent / 100))
        dim = (width, height)
        resized = cv2.resize(i, dim, interpolation = cv2.INTER_LANCZOS4)
        rescaled.append(resized)

    x_orig = np.reshape( rescaled, (len( rescaled), resized.shape[1], resized.shape[1], 3))

    return x_orig

In [6]:
def loadImages(path_data, path_labels):

    image_list = []
    

    for filename in tqdm.tqdm(sorted(glob.glob(path_data), key=natural_keys)): 
        im=cv2.imread(filename)
        #gray_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        imarray = np.array(im)
        #imarray = imarray.astype('float32')
        image_list.append(imarray)

    x_orig = np.reshape(image_list, (len(image_list), 90, 90, 3))
    
    path = path_labels    
    labels = pd.read_csv(path, usecols=["Type", "Category"],
                       sep="," )
    y_orig = np.array(labels['Category'])

    return x_orig, y_orig

In [7]:
p_data = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T60_T_97/3_class_Cont_ADR_HRH/'

train_data =  p_data + 'TRAIN/*.tiff'
train_lab= p_data + 'train_lab.csv'

validation_data = p_data + 'VALIDATION/*.tiff'
validation_lab = p_data + 'validation_lab.csv'


test_data= p_data + 'TEST/*.tiff'
test_lab= p_data + 'test_lab.csv'



In [8]:
x_train, y_train = loadImages(train_data, train_lab)
x_train.shape, y_train.shape

100%|██████████| 73354/73354 [01:05<00:00, 1127.09it/s]


((73354, 90, 90, 3), (73354,))

In [9]:
x_val, y_val = loadImages(validation_data, validation_lab)
x_val.shape, y_val.shape

100%|██████████| 19653/19653 [00:18<00:00, 1055.92it/s]


((19653, 90, 90, 3), (19653,))

In [10]:
x_test, y_test = loadImages(test_data, test_lab)
x_test.shape, y_test.shape

100%|██████████| 24106/24106 [00:24<00:00, 994.55it/s] 


((24106, 90, 90, 3), (24106,))

In [11]:
x_train = resize(x_train)
x_train.shape

(73354, 64, 64, 3)

In [12]:
x_val = resize(x_val)
x_val.shape

(19653, 64, 64, 3)

In [13]:
x_test = resize(x_test)
x_test.shape

(24106, 64, 64, 3)

In [14]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=3)

In [15]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 26564, 1: 36776, 2: 10014}

In [16]:
pretrained_model = VGG16(weights='imagenet',include_top=False, input_shape=(64, 64, 3))

base_model = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('block3_pool').output)

In [17]:
weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
print('weights = ' + str(weights))

weights = [0.9204688  0.66487202 2.44171493]


In [18]:
x_train = preprocess_input(x_train)

x_val = preprocess_input(x_val)

x_test = preprocess_input(x_test)


In [19]:
y_train = keras.utils.to_categorical(y_train)
y_test_1 = keras.utils.to_categorical(y_test)
y_val = keras.utils.to_categorical(y_val)

In [20]:
batch_size = 128

In [21]:
datagen = ImageDataGenerator()

datagen.fit(x_train)

train_gen = datagen.flow(x_train, y_train,batch_size=batch_size )

dat_val = ImageDataGenerator()

dat_val.fit(x_val)

val_gen = dat_val.flow(x_val, y_val,batch_size=batch_size)

m4 = Sequential()
m4.add(base_model)


m4.add(BatchNormalization())
m4.add(GlobalAveragePooling2D())
m4.add(BatchNormalization())
m4.add(Activation('relu'))
m4.add(Dense(64, activation='relu'))
m4.add(Dense(3,activation='softmax'))


base_model.trainable = False

opt = keras.optimizers.Adam(lr=1e-5)

m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])



epochs = 300

m4_h = m4.fit(train_gen,
                steps_per_epoch=(len(x_train)/batch_size),
                callbacks = [es],
                epochs=epochs,
                validation_data = (val_gen), 
                validation_steps = (len(x_val)/batch_size),
                class_weight = weights,
                 verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 573.078125 steps, validate for 153.5390625 steps
Epoch 1/300
574/573 [==============================] - 20s 35ms/step - loss: 1.1484 - accuracy: 0.3962 - val_loss: 1.0636 - val_accuracy: 0.4421
Epoch 2/300
574/573 [==============================] - 17s 29ms/step - loss: 1.0314 - accuracy: 0.4767 - val_loss: 1.0172 - val_accuracy: 0.4783
Epoch 3/300
574/573 [==============================] - 17s 29ms/step - loss: 0.9960 - accuracy: 0.4972 - val_loss: 0.9941 - val_accuracy: 0.4889
Epoch 4/300
574/573 [==============================] - 17s 29ms/step - loss: 0.9745 - accuracy: 0.5060 - val_loss: 0.9813 - val_accuracy: 0.4944
Epoch 5/300
574/573 [==============================] - 17s 29ms/step - loss: 0.9590 - accuracy: 0.5140 - val_loss: 0.9731 - val_accuracy: 0.4988
Epoch 6/300
574/573 [==============================] - 17s 29ms/step - loss: 0.9471 - accuracy: 0.5214 - val_loss: 0.9673 - val_accuracy: 0.5001
Epoch 7/300
574/573 [

In [22]:
base_model.trainable = True

opt = keras.optimizers.Adam(lr=1e-5)

m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])

epochs = 300

m4_h = m4.fit(train_gen,
                steps_per_epoch=(len(x_train)/batch_size),
                callbacks = [es],
                epochs=epochs,
                validation_data = val_gen, 
                validation_steps = (len(x_val)/batch_size),
                class_weight = weights,
                verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 573.078125 steps, validate for 153.5390625 steps
Epoch 1/300
574/573 [==============================] - 39s 68ms/step - loss: 0.7563 - accuracy: 0.6517 - val_loss: 0.9220 - val_accuracy: 0.5318
Epoch 2/300
574/573 [==============================] - 37s 65ms/step - loss: 0.7026 - accuracy: 0.6876 - val_loss: 0.8839 - val_accuracy: 0.5484
Epoch 3/300
574/573 [==============================] - 38s 66ms/step - loss: 0.6614 - accuracy: 0.7141 - val_loss: 0.8798 - val_accuracy: 0.5655
Epoch 4/300
574/573 [==============================] - 38s 66ms/step - loss: 0.6267 - accuracy: 0.7341 - val_loss: 0.9077 - val_accuracy: 0.5444
Epoch 5/300
574/573 [==============================] - 38s 65ms/step - loss: 0.5951 - accuracy: 0.7528 - val_loss: 0.8773 - val_accuracy: 0.5683
Epoch 6/300
574/573 [==============================] - 37s 65ms/step - loss: 0.5654 - accuracy: 0.7707 - val_loss: 0.8703 - val_accuracy: 0.5835
Epoch 7/300
574/573 [

In [ ]:
plot_history(m4_h, 'first')

In [ ]:
scores = m4.evaluate(x_test, y_test_1, verbose = 0)
print("\n%s: %.2f%%" % (m4.metrics_names[1], scores[1]*100))


In [ ]:
test_preds = m4.predict(x_test)

preds_df = pd.DataFrame(test_preds)
predicted_labels = preds_df.idxmax(axis=1)

In [ ]:
draw_confusion_matrix(y_test, predicted_labels)

In [ ]:
print(classification_report(y_test, predicted_labels))

In [ ]:
#p = '/home/jovyan/Saved_Models/'
#m4.save(p + "CNN_same_fields_view_Controll_HRH_ADR.h5")